### 유튜브 영상 댓글 크롤링 ver3
- 영상 자막 스크립트 + 댓글

In [ ]:
# ######### 신버전 ##########
# from youtube_transcript_api import YouTubeTranscriptApi

# video_id = "lxZsn3cwdAU"

# api = YouTubeTranscriptApi()
# transcription = api.fetch(video_id, languages=['ko', 'en'])

# for seg in transcription:
#     print(seg.text)


In [ ]:
import requests
import pandas as pd
from time import sleep
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np
from konlpy.tag import Okt
import os
import re
from youtube_transcript_api import YouTubeTranscriptApi

# ================================
# 0) API 키 & 수집할 URL 설정
# ================================
API_KEY = ""  # 🔹 여기에 YouTube Data API 키 넣기

urls = [
    "https://www.youtube.com/shorts/lxZsn3cwdAU",
    "https://www.youtube.com/watch?v=dFRXcYJVfMM",
]

# 결과 저장 폴더 설정
RESULT_DIR = "youtube_results_commentsADDscript"
os.makedirs(RESULT_DIR, exist_ok=True)


# ================================
# 1) URL에서 video_id 추출 함수
# ================================
def extract_video_id(url: str):
    """
    유튜브 일반 영상 & 쇼츠 URL에서 video_id만 뽑아주는 함수
    """
    # 쿼리파라미터(?) 제거용
    base = url.split("?")[0]

    # shorts 형식: https://www.youtube.com/shorts/VIDEO_ID
    if "shorts/" in base:
        return base.split("shorts/")[1]

    # 일반 영상 형식: https://www.youtube.com/watch?v=VIDEO_ID
    if "watch" in base and "v=" in url:
        return url.split("v=")[1].split("&")[0]

    return None


# ================================
# 2) 특정 video_id 전체 댓글 수집
# ================================
def fetch_all_comments_for_video(video_id: str, sleep_sec: float = 0.2):
    """
    특정 video_id에 대한 상위 댓글(Top-level comments)을
    마지막 페이지까지 전부 수집하는 함수
    """
    url = "https://www.googleapis.com/youtube/v3/commentThreads"
    params = {
        "part": "snippet",
        "videoId": video_id,
        "maxResults": 100,           # 한 페이지 최대 100개
        "textFormat": "plainText",
        "key": API_KEY,
    }

    comments = []
    next_page_token = None
    page_idx = 0

    while True:
        if next_page_token:
            params["pageToken"] = next_page_token

        res = requests.get(url, params=params)
        data = res.json()

        # 에러 발생 시 출력하고 중단
        if "error" in data:
            print(f"⚠️ API Error for video {video_id}: ", data["error"])
            break

        items = data.get("items", [])
        if not items:
            break

        for item in items:
            snippet = item["snippet"]["topLevelComment"]["snippet"]
            comments.append({
                "video_id": video_id,
                "source": "comment",
                "comment_id": item["id"],
                "author": snippet.get("authorDisplayName"),
                "text": snippet.get("textDisplay"),
                "likeCount": snippet.get("likeCount"),
                "publishedAt": snippet.get("publishedAt"),
            })

        page_idx += 1
        print(f"✅ {video_id} - {page_idx} 페이지 수집 완료, 누적 댓글 수: {len(comments)}")

        # 다음 페이지 토큰 확인
        next_page_token = data.get("nextPageToken")
        if not next_page_token:
            break

        sleep(sleep_sec)  # API 호출 간 살짝 쉬어주기 (쿼터/차단 방지)

    return comments


# ================================
# 2-1) 특정 video_id 스크립트(자막) 수집
# ================================
def fetch_full_transcript(video_id: str, languages=None) -> str | None:
    """
    youtube_transcript_api의 최신 API (api.fetch) 방식으로
    전체 스크립트를 하나의 문자열로 합쳐서 반환.
    """
    if languages is None:
        languages = ['ko', 'en']  # 한국어 > 영어 우선

    try:
        api = YouTubeTranscriptApi()

        # 🔹 신버전: fetch() 사용
        segments = api.fetch(video_id, languages=languages)

        # 🔹 자막 텍스트만 합치기
        full_text = " ".join(seg.get("text", "") for seg in segments if seg.get("text"))

        print(f"📜 {video_id} 스크립트 길이: {len(full_text)}자")

        if not full_text.strip():
            print(f"⚠️ {video_id} 스크립트 비어있음")
            return None

        return full_text

    except Exception as e:
        print(f"⚠️ Transcript fetch 실패 ({video_id}): {e}")
        return None


# ================================
# 3) 모든 URL에 대해 스크립트 + 댓글 수집
# ================================
def collect_data_from_urls(url_list):
    video_ids = [extract_video_id(u) for u in url_list]
    print("🎬 추출된 video_id 목록:", video_ids)

    all_rows = []  # 전체 영상 통합용

    for vid in video_ids:
        if vid is None:
            print(f"⚠️ URL에서 video_id를 추출하지 못했습니다. 건너뜀.")
            continue

        print(f"\n📥 {vid} 데이터 수집 시작")

        per_video_rows = []  # 이 영상 한 개에 대한 행들만

        # 1) 스크립트(자막) 수집
        script_text = fetch_full_transcript(vid)
        if script_text:
            script_row = {
                "video_id": vid,
                "source": "script",   # 🔹 스크립트
                "comment_id": None,
                "author": None,
                "text": script_text,
                "likeCount": None,
                "publishedAt": None,
            }
            per_video_rows.append(script_row)
            all_rows.append(script_row)
            print(f"✅ {vid} 스크립트 1행 추가 완료")

        # 2) 댓글 수집
        print(f"💬 {vid} 댓글 전체 수집 시작")
        video_comments = fetch_all_comments_for_video(vid)

        if video_comments:
            # 이미 source, video_id, text 등 포함되어 있음
            per_video_rows.extend(video_comments)
            all_rows.extend(video_comments)
            print(f"✅ {vid} 댓글 {len(video_comments)}개 수집 완료")
        else:
            print(f"⚠️ 동영상 {vid} 에는 댓글이 없습니다.")

        # 3) 이 영상 전용 CSV 저장 (스크립트 + 댓글)
        if per_video_rows:
            df_vid = pd.DataFrame(per_video_rows)
            filename = f"video_{vid}_script_comments.csv"
            path = os.path.join(RESULT_DIR, filename)
            df_vid.to_csv(path, index=False, encoding="utf-8-sig")
            print(f"💾 (영상별 저장) 동영상 {vid} CSV 저장 완료: {path}")
        else:
            print(f"⚠️ 동영상 {vid} 에서 수집된 데이터가 없습니다.")

    # 모든 영상 합친 DataFrame
    df_all = pd.DataFrame(all_rows)
    return df_all


# ================================
# 4) CSV & PKL 저장 (전체 통합)
# ================================
def save_comments(df: pd.DataFrame,
                  csv_name="youtube_all_videos_script_comments.csv",
                  pkl_name="youtube_all_videos_script_comments.pkl"):
    csv_path = os.path.join(RESULT_DIR, csv_name)
    pkl_path = os.path.join(RESULT_DIR, pkl_name)

    df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    df.to_pickle(pkl_path)

    print(f"💾 CSV 저장 완료: {csv_path}")
    print(f"💾 PKL 저장 완료: {pkl_path}")


# ================================
# 4-2) 한국어 전처리 + 명사/형용사 추출
# ================================
okt = Okt()

def clean_text(text: str) -> str:
    """
    기본 전처리:
    - URL 제거
    - 이모지 제거
    - 특수문자 제거
    - 영어/숫자 제거
    - 중복 공백 정리
    """
    # 1) URL 제거
    text = re.sub(r'http\S+|www\.\S+', '', text)

    # 2) 이모지 제거 (유니코드 상위 영역 날리기)
    text = re.sub(r'[\U00010000-\U0010ffff]', '', text)

    # 3) 특수문자 제거 (한글/영문/숫자/공백만 남기기)
    text = re.sub(r'[^가-힣a-zA-Z0-9\s]', ' ', text)

    # 4) 영어/숫자 제거 (한국어 위주 분석이면 추천)
    text = re.sub(r'[a-zA-Z0-9]', ' ', text)

    # 5) 중복 공백 제거
    text = re.sub(r'\s+', ' ', text).strip()

    return text


def extract_korean_adjectives(texts):
    """
    한국어 텍스트 리스트에서 형용사(Adjective)만 뽑아서
    하나의 큰 문자열로 반환 (전처리 포함)
    """
    adjectives = []

    for t in texts:
        try:
            cleaned = clean_text(t)  # 🔥 전처리 먼저!
            pos_tags = okt.pos(cleaned, norm=True, stem=True)
            for word, tag in pos_tags:
                if tag == "Adjective":
                    adjectives.append(word)
        except Exception:
            continue

    return " ".join(adjectives)


def extract_korean_nouns(texts):
    """
    한국어 텍스트 리스트에서 명사(Noun)만 추출하여
    하나의 큰 문자열로 반환 (전처리 포함)
    """
    nouns = []

    for t in texts:
        try:
            cleaned = clean_text(t)  # 🔥 전처리 먼저
            pos_tags = okt.pos(cleaned, norm=True, stem=True)
            for word, tag in pos_tags:
                if tag == "Noun":
                    nouns.append(word)
        except Exception:
            continue

    return " ".join(nouns)


# ================================
# 5) 워드 클라우드 생성 (스크립트+댓글 전체 대상)
# ================================
def create_wordcloud(df: pd.DataFrame,
                     output_img: str = "youtube_comments_wordcloud.png",
                     mask_path: str | None = None):
    # 1) 텍스트 모으기 (스크립트 + 댓글 모두)
    texts = df["text"].dropna().astype(str).tolist()
    if not texts:
        print("⚠️ 생성할 텍스트가 없습니다.")
        return

    # 🔹 명사 / 형용사 각각 전처리 및 추출
    print("🔎 명사 추출 중...")
    nouns_text = extract_korean_nouns(texts)

    print("🔎 형용사 추출 중...")
    adjs_text = extract_korean_adjectives(texts)

    # 3) 마스크 적용
    mask = None
    if mask_path is not None and os.path.exists(mask_path):
        mask = np.array(plt.imread(mask_path))
        print(f"🎭 마스크 적용: {mask_path}")
    elif mask_path is not None:
        print(f"⚠️ 마스크 파일을 찾을 수 없습니다: {mask_path} (마스크 없이 생성)")

    # 4) 폰트 경로 설정 (환경 맞게 수정!)
    #   - Mac: '/Library/Fonts/AppleGothic.ttf'
    #   - Win: 'C:/Windows/Fonts/malgun.ttf'
    font_path = "/Library/Fonts/AppleGothic.ttf"

    # =======
    # 1) 명사 워드클라우드 생성
    # ======
    wc_noun = WordCloud(
        font_path=font_path,
        width=1000,
        height=800,
        background_color="white",
        mask=mask
    ).generate(nouns_text)

    noun_img_path = os.path.join(RESULT_DIR, "wordcloud_nouns.png")
    plt.figure(figsize=(10, 6))
    plt.imshow(wc_noun)
    plt.axis("off")
    plt.tight_layout()
    plt.savefig(noun_img_path, dpi=300)
    print(f"🖼 명사 워드클라우드 저장 완료: {noun_img_path}")

    # ========
    # 2) 형용사 워드클라우드 생성
    # ========
    wc_adj = WordCloud(
        font_path=font_path,
        width=1000,
        height=800,
        background_color="white",
        mask=mask
    ).generate(adjs_text)

    adj_img_path = os.path.join(RESULT_DIR, "wordcloud_adjectives.png")
    plt.figure(figsize=(10, 6))
    plt.imshow(wc_adj)
    plt.axis("off")
    plt.tight_layout()
    plt.savefig(adj_img_path, dpi=300)
    print(f"🖼 형용사 워드클라우드 저장 완료: {adj_img_path}")


# ================================
# 6) 메인 실행부
# ================================
if __name__ == "__main__":
    # 1) 스크립트 + 댓글 전체 수집
    df_all = collect_data_from_urls(urls)
    print("\n📊 수집된 데이터프레임 미리보기:")
    print(df_all.head())

    print("------------csv, pkl, wordcloud--------------")
    # 2) CSV / PKL 저장 (모든 영상 통합)
    save_comments(df_all)

    # 3) 워드클라우드 생성 (스크립트 + 댓글 전체 대상)
    create_wordcloud(
        df_all,
        mask_path="heart.jpg",
    )
